# Esercizio 2 — Filtri e Feature Engineering (propedeutico)

**Obiettivi:**
- Usare filtri e loc/boolean indexing per selezionare righe;
- Creare nuove feature a partire da colonne esistenti;
- Ordinare e ispezionare i risultati.


## Setup: import e `DataFrame` di esempio
- `pd.DataFrame({...})` costruisce una tabella da un dizionario: le chiavi diventano colonne e le liste righe.
- `import pandas as pd` e `import numpy as np` creano gli alias `pd` e `np`, usati in tutte le funzioni successive (es. `pd.DataFrame`, `np.mean`).
La cella finale mostra `df` perché in Jupyter l'ultima istruzione restituisce implicitamente il valore della variabile.


In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    "Giorno": ["Lun", "Mar", "Mer", "Gio", "Ven", "Sab", "Dom"],
    "Temperatura": [18, 20, 21, 19, 23, 25, 22],
    "Umidita": [60, 55, 58, 63, 50, 45, 52],
    "Pioggia": [0, 1, 0, 1, 0, 0, 1]
})

df

,Giorno,Temperatura,Umidita,Pioggia
0,Lun,18,60,0
1,Mar,20,55,1
2,Mer,21,58,0
3,Gio,19,63,1
4,Ven,23,50,0
5,Sab,25,45,0
6,Dom,22,52,1


## Filtri multipli e selezione righe
Funzioni e sintassi principali:
- `df["Temperatura"].mean()` calcola la media della colonna (metodo `Series.mean()`);
- `df["Pioggia"] == 0` produce una serie booleana;
- Le condizioni si combinano con `&` (AND) e `|` (OR) perché lavoriamo con array/serie;
- `df[condizione]` applica il boolean indexing restituendo solo le righe `True`;
- `df.loc[condizione, col_subset]` permette di specificare anche le colonne da visualizzare (qui solo `"Giorno"`).


In [2]:
df_temp =  df[(df["Temperatura"] > df["Temperatura"].mean()) & (df["Pioggia"] == 0)]
df_temp

,Giorno,Temperatura,Umidita,Pioggia
4,Ven,23,50,0
5,Sab,25,45,0


In [3]:
df_temp =  df.loc[(df["Temperatura"] > df["Temperatura"].mean())
                   & (df["Pioggia"] == 0), ["Giorno"]]
df_temp

,Giorno
4,Ven
5,Sab


## Filtrare con condizioni multiple (esempio avanzato)
- `df["Umidita"].mean()` fornisce la media dell'umidità;
- `df["Temperatura"].median()` restituisce la mediana (metodo `Series.median()`);
- Anche qui usiamo `df.loc[condizione, ["colonne"]]` per ottenere un sottoinsieme ordinato delle colonne utili.
Questo pattern è ricorrente nei filtri complessi.


In [ ]:
df_umid = df.loc[(df["Umidita"] < df["Umidita"].mean()) & 
                 (df["Temperatura"] >= df["Temperatura"].median()), ["Giorno", "Temperatura"]]
df_umid

,Giorno,Temperatura
4,Ven,23
5,Sab,25
6,Dom,22


## Feature Engineering: creare nuove colonne
- `df["Indice_Comfort"] = ...` crea una nuova colonna direttamente; l'espressione `df["Temperatura"] - (df["Umidita"] / 8)` sfrutta l'algebra vettoriale riga-per-riga.
- `df["Umidita"].quantile(0.25)` calcola il 25° percentile (primo quartile) della colonna; confrontarlo con ogni valore permette di definire `Molto_Secco` come booleano.
Ricorda che tutte le operazioni si applicano elemento per elemento grazie al broadcasting di pandas/NumPy.


In [13]:
"Comfort_Index=Temperatura−8Umidita​"

'Comfort_Index=Temperatura−8Umidita\u200b'

In [14]:
df["Indice_Comfort"] = df["Temperatura"] - (df["Umidita"] / 8)
df

,Giorno,Temperatura,Umidita,Pioggia,Indice_Comfort
0,Lun,18,60,0,10.500
1,Mar,20,55,1,13.125
2,Mer,21,58,0,13.750
3,Gio,19,63,1,11.125
4,Ven,23,50,0,16.750
5,Sab,25,45,0,19.375
6,Dom,22,52,1,15.500


In [15]:
df["Molto_Secco"] = (df["Umidita"] < df["Umidita"].quantile(0.25))
df

,Giorno,Temperatura,Umidita,Pioggia,Indice_Comfort,Molto_Secco
0,Lun,18,60,0,10.500,False
1,Mar,20,55,1,13.125,False
2,Mer,21,58,0,13.750,False
3,Gio,19,63,1,11.125,False
4,Ven,23,50,0,16.750,True
5,Sab,25,45,0,19.375,True
6,Dom,22,52,1,15.500,False


## Ordinare i risultati
`df.sort_values(colonna, ascending=False)` ordina il `DataFrame` in base alla colonna scelta. Parametri principali:
- `by` (qui implicito perché primo argomento posizionale) indica la colonna o la lista di colonne;
- `ascending=False` impone l'ordine decrescente (True sarebbe crescente).
Il metodo restituisce una copia ordinata, quindi è perfetto per analisi rapide.


In [16]:
df.sort_values("Indice_Comfort", ascending= False)

,Giorno,Temperatura,Umidita,Pioggia,Indice_Comfort,Molto_Secco
5,Sab,25,45,0,19.375,True
4,Ven,23,50,0,16.750,True
6,Dom,22,52,1,15.500,False
2,Mer,21,58,0,13.750,False
1,Mar,20,55,1,13.125,False
3,Gio,19,63,1,11.125,False
0,Lun,18,60,0,10.500,False


## Riepilogo funzioni e operazioni del notebook
Obiettivo: chiarire ogni metodo / sintassi usata.

### Creazione dati
- `pd.DataFrame({...})`: chiavi del dizionario = nomi colonne; liste = valori per riga. Tutte le liste devono avere stessa lunghezza.

### Accesso a colonne
- `df['Colonna']`: restituisce una `Series`.
- Chaining: `df['Temperatura'].mean()` → media dei valori numerici.

### Statistiche semplici
- `.mean()`: media aritmetica. 
- (Eventuali `.std()`, `.median()`, `.quantile(q)`) seguono lo stesso pattern: metodo su `Series`.

### Confronti e logica booleana
- `df['Pioggia'] == 0`: array booleano (True dove il valore è 0).
- Operatori su array: `&` = AND, `|` = OR, `~` = NOT. Devono essere racchiusi tra parentesi: `(cond1 & cond2)`.

### Boolean indexing
- `df[condizione]`: filtra le righe con `True`.
- `df.loc[condizione, ['Giorno']]`: filtra e seleziona solo le colonne indicate. Con `:` al posto della lista si mantengono tutte le colonne.

### Creazione / modifica colonne
- `df['Nuova'] = espressione`: assegna (crea o sovrascrive) la colonna. L'espressione deve avere lunghezza compatibile (stessa lunghezza delle righe) oppure essere uno scalare.
Esempi tipici nel notebook:
- `Indice_Comfort = Temperatura * 0.7 + (100 - Umidita) * 0.3` (combinazione lineare).
- `Molto_Secco = df['Umidita'] < 52` → Series booleana (True/False).

### Ordinamento
- `df.sort_values('Indice_Comfort', ascending=False)`: ordina le righe per la colonna. 
  - `ascending=False` ⇒ dal valore più alto al più basso.
  - Restituisce un nuovo DataFrame (non modifica in-place a meno di `inplace=True`).

### Tipi di dato impliciti
- Operazioni aritmetiche tra `int` e `float` promuovono a `float`.
- Colonne booleane possono essere usate direttamente nei filtri.

### Interpretazione
- Il calcolo di un *indice* combinando temperatura e umidità è un esempio di feature engineering: creare variabili che riassumono condizioni multiple.
- La colonna `Molto_Secco` facilita filtri successivi senza ripetere la condizione.

### Buone pratiche
- Racchiudi ogni condizione in parentesi prima di combinare con `&` / `|`.
- Evita di usare `.loc` con maschera e senza colonne se devi solo filtrare: `df[maschera]` è più breve. Usa `.loc` quando selezioni anche colonne o devi assegnare.
- Usa nomi colonna descrittivi (evita abbreviazioni ambigue).

Ora ogni riga di codice dovrebbe essere leggibile e motivata.
